In [1]:
import requests
import pandas as pd
import time

In [2]:
!pip install git+https://github.com/traveltime-dev/traveltime-python-sdk

  Cloning https://github.com/traveltime-dev/traveltime-python-sdk to /private/var/folders/s0/2b77knw54rbfdpksvcc04_j00000gn/T/pip-req-build-7qtf2sh4
  Running command git clone --filter=blob:none --quiet https://github.com/traveltime-dev/traveltime-python-sdk /private/var/folders/s0/2b77knw54rbfdpksvcc04_j00000gn/T/pip-req-build-7qtf2sh4
  Resolved https://github.com/traveltime-dev/traveltime-python-sdk to commit 26d69d9cefe5feb099e9cdeb991bf12ed2c859cf
  Preparing metadata (setup.py) ... done


In [3]:
import traveltimepy as ttpy
from datetime import datetime
import os

In [4]:
os.environ["TRAVELTIME_ID"] = os.environ.get("TRAVELTIME_ID")
os.environ["TRAVELTIME_KEY"] = os.environ.get("TRAVELTIME_KEY")

In [15]:
locations_list = [
    {"id": "London center", "coords": {"lat": 51.508930, "lng": -0.131387}},
    {"id": "Hyde Park", "coords": {"lat": 51.508824, "lng": -0.167093}},
    {"id": "ZSL London Zoo", "coords": {"lat": 51.536067, "lng": -0.153596}},
    {"id": "London Museum", "coords": {"lat": 51.520464 , "lng": -0.126312}},
    {"id": "Big Ben", "coords": {"lat": 51.500896 , "lng": -0.124584}},
    {"id": "Charing Cross Station", "coords": {"lat": 51.508567 , "lng": -0.125633}},
    {"id": "London center2", "coords": {"lat": 51.508930, "lng": -0.131387}},
    {"id": "Hyde Park2", "coords": {"lat": 51.508824, "lng": -0.167093}},
    {"id": "ZSL London Zoo2", "coords": {"lat": 51.536067, "lng": -0.153596}},
    {"id": "London Museum2", "coords": {"lat": 51.520464 , "lng": -0.126312}},
    {"id": "Big Ben2", "coords": {"lat": 51.500896 , "lng": -0.124584}},
    {"id": "Charing Cross Station2", "coords": {"lat": 51.508567 , "lng": -0.125633}}
]

In [16]:
def get_results_from_api(locs):
    
    """
    Calls the TravelTime API and returns the results in a dictionary.
    
    Args: 
      locs (list): A list of dictionaries containing locations. Must contain keys 
        ["id", "coords"].
      dep_search (dict): A dictionary of departure locations. Must contain keys 
        ["id", "departure_location_id", "arrival_location_ids", "transportation", 
        "departure_time", "travel_time", "properties", "range"]

    Returns: 
      dict: A dictionary of results.   

    """
    
    departure = locs[0]["id"]
    arrival_locations = [locs[index]["id"] for index in range(len(locs)) 
                         if locs[index]["id"]!= departure]
    
    # Define parameters in dictionary for API call
    
    public_parameters = {
    "id": "arrive-at one-to-many search example",
    "arrival_location_ids": arrival_locations,
    "departure_location_id": departure,
    "transportation": {"type": "public_transport"},
    "arrival_time_period": "weekday_morning",
    "travel_time": 3600,
    "properties": ["travel_time"]
    }
    
    private_parameters = {
    "id": "arrive-at one-to-many search example",
    "arrival_location_ids": arrival_locations,
    "departure_location_id": departure,
    "transportation": {"type": "driving"},
    "arrival_time_period": "weekday_morning",
    "travel_time": 3600,
    "properties": ["travel_time"]
    }

    # Call the API 
    public_api_data = ttpy.time_filter_fast(locations=locs, arrival_one_to_many=public_parameters)

    private_api_data = ttpy.time_filter_fast(locations=locs, arrival_one_to_many=private_parameters)

    API_call_time = time.ctime()
    
    # Define empty results dictionary to store results
    res_dict = {"Start" : [], "Destination" : [], "Public_Travel_Duration" : [], "Private_Travel_Duration" : [], "API call time" : []}
    # number_of_arrival_locations = len(arrival_locations)

    public_refined_api_data = public_api_data["results"][0]["locations"]
    private_refined_api_data = private_api_data["results"][0]["locations"]

    # Loop through destination locations and store each journey time into results dictionary.        
    for destination_result in public_refined_api_data:
        public_duration_result = destination_result["properties"]["travel_time"]
        destination_name = destination_result["id"]
        res_dict["Start"].append(departure)
        res_dict["Destination"].append(destination_name)
        res_dict["Public_Travel_Duration"].append(public_duration_result)
        res_dict["API call time"].append(API_call_time)
    
    for destination_result in private_refined_api_data:
        private_duration_result = destination_result["properties"]["travel_time"]
        res_dict["Private_Travel_Duration"].append(private_duration_result)

    final_df = pd.DataFrame(res_dict)
    
    return final_df

In [17]:
api_output_df = get_results_from_api(locs=locations_list)
api_output_df

,Start,Destination,Public_Travel_Duration,Private_Travel_Duration,API call time
0,London center,Charing Cross Station2,432,277,Mon Mar 28 16:58:46 2022
1,London center,Hyde Park,1527,1040,Mon Mar 28 16:58:46 2022
2,London center,Big Ben2,632,555,Mon Mar 28 16:58:46 2022
3,London center,ZSL London Zoo,1890,1026,Mon Mar 28 16:58:46 2022
4,London center,London Museum,786,557,Mon Mar 28 16:58:46 2022
5,London center,ZSL London Zoo2,1890,1026,Mon Mar 28 16:58:46 2022
6,London center,Big Ben,632,555,Mon Mar 28 16:58:46 2022
7,London center,London center2,40,40,Mon Mar 28 16:58:46 2022
8,London center,Hyde Park2,1527,1040,Mon Mar 28 16:58:46 2022
9,London center,Charing Cross Station,432,277,Mon Mar 28 16:58:46 2022


In [20]:
departure = locations_list[0]["id"]
arrival_locations = [locations_list[index]["id"] for index in range(len(locations_list)) 
                     if locations_list[index]["id"]!= departure]

# Define parameters in dictionary for API call

public_parameters = {
"id": "arrive-at one-to-many search example",
"arrival_location_ids": arrival_locations,
"departure_location_id": departure,
"transportation": {"type": "public_transport"},
"arrival_time_period": "weekday_morning",
"travel_time": 3600,
"properties": ["travel_time"]
}

private_parameters = {
"id": "arrive-at one-to-many search example",
"arrival_location_ids": arrival_locations,
"departure_location_id": departure,
"transportation": {"type": "driving"},
"arrival_time_period": "weekday_morning",
"travel_time": 3600,
"properties": ["travel_time"]
}

# Call the API 
public_api_data = ttpy.time_filter_fast(locations=locations_list, arrival_one_to_many=public_parameters)

private_api_data = ttpy.time_filter_fast(locations=locations_list, arrival_one_to_many=private_parameters)

API_call_time = time.ctime()

# Define empty results dictionary to store results
res_dict = {"Start" : [], "Destination" : [], "Public_Travel_Duration" : [], "Private_Travel_Duration" : [], "API call time" : []}
# number_of_arrival_locations = len(arrival_locations)

public_refined_api_data = public_api_data["results"][0]["locations"]
private_refined_api_data = private_api_data["results"][0]["locations"]

# Loop through destination locations and store each journey time into results dictionary.        
for destination_result in public_refined_api_data:
    public_duration_result = destination_result["properties"]["travel_time"]
    destination_name = destination_result["id"]
    res_dict["Start"].append(departure)
    res_dict["Destination"].append(destination_name)
    res_dict["Public_Travel_Duration"].append(public_duration_result)
    res_dict["API call time"].append(API_call_time)

for destination_result in private_refined_api_data:
    private_duration_result = destination_result["properties"]["travel_time"]
    res_dict["Private_Travel_Duration"].append(private_duration_result)

final_df = pd.DataFrame(res_dict)

In [73]:
public_api_data

{'results': [{'search_id': 'arrive-at one-to-many search example',
   'locations': [{'id': 'Charing Cross Station2',
     'properties': {'travel_time': 432}},
    {'id': 'London center2', 'properties': {'travel_time': 40}},
    {'id': 'Charing Cross Station', 'properties': {'travel_time': 432}}],
   'unreachable': ['Big Ben2',
    'ZSL London Zoo',
    'London Museum',
    'ZSL London Zoo2',
    'London Museum2',
    'Hyde Park',
    'Big Ben',
    'Hyde Park2']}]}

In [75]:
public_api_data_refined = public_api_data["results"][0]["locations"]
public_api_data_refined

[{'id': 'Charing Cross Station2', 'properties': {'travel_time': 432}},
 {'id': 'London center2', 'properties': {'travel_time': 40}},
 {'id': 'Charing Cross Station', 'properties': {'travel_time': 432}}]

In [ ]:
for location in arrival_locations:
    for dict_result in public_api_data_refined:
        if dict_result["id"] == location:
            append

In [68]:
private_api_data

TypeError: 'dict' object is not callable

In [ ]:
private_api_data

In [44]:
res_dict_idea = {"Start" : [], "Destination" : [], "Public_Travel_Duration" : [], "Private_Travel_Duration" : [], "API call time" : []}

In [24]:
public_refined_api_data = public_api_data["results"][0]["locations"]
private_refined_api_data = private_api_data["results"][0]["locations"]

In [62]:
for number in range(len(public_refined_api_data):
                    

{'id': 'Charing Cross Station2', 'properties': {'travel_time': 432}}

In [28]:
arrival_locations

['Hyde Park',
 'ZSL London Zoo',
 'London Museum',
 'Big Ben',
 'Charing Cross Station',
 'London center2',
 'Hyde Park2',
 'ZSL London Zoo2',
 'London Museum2',
 'Big Ben2',
 'Charing Cross Station2']

In [33]:
for location in arrival_locations:
    print(location)

Hyde Park
ZSL London Zoo
London Museum
Big Ben
Charing Cross Station
London center2
Hyde Park2
ZSL London Zoo2
London Museum2
Big Ben2
Charing Cross Station2


In [ ]:
for location in arrival_locations:
    print(location)

In [45]:
for location in arrival_locations:
    public_duration_result = destination_result["properties"]["travel_time"]
    destination_name = destination_result["id"]
    res_dict_idea["Start"].append(departure)
    res_dict_idea["Destination"].append(location)
    res_dict_idea["Public_Travel_Duration"].append(location)

In [46]:
res_dict_idea

{'Start': ['London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center',
  'London center'],
 'Destination': ['Hyde Park',
  'ZSL London Zoo',
  'London Museum',
  'Big Ben',
  'Charing Cross Station',
  'London center2',
  'Hyde Park2',
  'ZSL London Zoo2',
  'London Museum2',
  'Big Ben2',
  'Charing Cross Station2'],
 'Public_Travel_Duration': [],
 'Private_Travel_Duration': [],
 'API call time': []}

In [69]:
public_parameters = {
"id": "arrive-at one-to-many search example",
"arrival_location_ids": arrival_locations,
"departure_location_id": departure,
"transportation": {"type": "public_transport"},
"arrival_time_period": "weekday_morning",
"travel_time": 500,
"properties": ["travel_time"]
}

# Call the API 
public_api_data = ttpy.time_filter_fast(locations=locations_list, arrival_one_to_many=public_parameters)

In [70]:
public_api_data

{'results': [{'search_id': 'arrive-at one-to-many search example',
   'locations': [{'id': 'Charing Cross Station2',
     'properties': {'travel_time': 432}},
    {'id': 'London center2', 'properties': {'travel_time': 40}},
    {'id': 'Charing Cross Station', 'properties': {'travel_time': 432}}],
   'unreachable': ['Big Ben2',
    'ZSL London Zoo',
    'London Museum',
    'ZSL London Zoo2',
    'London Museum2',
    'Hyde Park',
    'Big Ben',
    'Hyde Park2']}]}